# **💁🏻🗨️💁🏻‍♂️대화 요약 Baseline code**
> t5 모델 baseline
- T5TokenizerFast, T5ForConditionalGeneration, T5Config 사용
- 입출력 데이터 전처리
    - DatasetForTrain, DatasetForVal : item[input_ids], item[attention_mask], item[labels]만 반환
    - Processor 클래스의 make_input 메소드 : dialogue에 prefix 추가

## ⚙️ 데이터 및 환경설정

### 1) 필요한 라이브러리 설치

- 필요한 라이브러리를 설치한 후 불러옵니다.

In [1]:
import pandas as pd
import os
import re
import json
import yaml
from glob import glob
from tqdm import tqdm
from pprint import pprint
import torch
import pytorch_lightning as pl
from rouge import Rouge # 모델의 성능을 평가하기 위한 라이브러리입니다.

from torch.utils.data import Dataset , DataLoader
# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoConfig
# from transformers import BartForConditionalGeneration, BartConfig
# from transformers import T5TokenizerFast, T5ForConditionalGeneration, T5Config
# from transformers import Trainer, TrainingArguments
from transformers import PegasusForConditionalGeneration, PegasusTokenizer, PegasusConfig
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import EarlyStoppingCallback

import wandb # 모델 학습 과정을 손쉽게 Tracking하고, 시각화할 수 있는 라이브러리입니다.

2024-09-10 23:09:14.083712: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-10 23:09:14.097513: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-10 23:09:14.101757: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-10 23:09:14.111648: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-10 23:09:14.735900: W tensorflow/compiler/tf2

### 2) Config file 만들기 (선택)
- 모델 생성에 필요한 다양한 매개변수 정보를 저장할 수 있습니다.  
  따라서, 코드 상에서 모델의 매개변수를 설정할 수도 있지만 독립적인 매개변수 정보 파일을 생성하여 관리할 수 있습니다.

In [2]:
model_name = "google/pegasus-xsum"
# paust/pko-t5-base
# lcw99/t5-base-korean-text-summary
# noahkim/KoT5_news_summarization
# lcw99/t5-large-korean-text-summary

# config 설정에 tokenizer 모듈이 사용되므로 미리 tokenizer를 정의해줍니다.
tokenizer = PegasusTokenizer.from_pretrained(model_name)

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [7]:
config_data = {
    "general": {
        "data_path": "../data_en/", # 모델 생성에 필요한 데이터 경로를 사용자 환경에 맞게 지정합니다.
        "model_name": f"{model_name}", # 불러올 모델의 이름을 사용자 환경에 맞게 지정할 수 있습니다.
        "output_dir": "./" # 모델의 최종 출력 값을 저장할 경로를 설정합니다.
    },
    "tokenizer": {
        "encoder_max_len": 512, # 1000
        "decoder_max_len": 100, # 200
        "bos_token": f"{tokenizer.bos_token}",
        "eos_token": f"{tokenizer.eos_token}",
        # 특정 단어들이 분해되어 tokenization이 수행되지 않도록 special_tokens을 지정해줍니다.
        "special_tokens": ['#Person1#', '#Person2#', '#Person3#', '#PhoneNumber#', '#Address#', '#PassportNumber#',
                           '#DateOfBirth#', '#SSN#', '#CardNumber#', '#CarNumber#', '#Email#',
                           '#Person#', '#Person4#', '#Person5#', '#Person6#', '#Person7#']
    },
    "training": {
        "overwrite_output_dir": True,
        "num_train_epochs": 20, # 원래 20,50
        "learning_rate": 1e-5, # 1e-5, optuna: 3e-5
        "per_device_train_batch_size": 16, # 원래 1로 되어 있었고, Bart에서는 16
        "per_device_eval_batch_size": 32, # 원래  1,32
        "warmup_ratio": 0.1,
        "weight_decay": 0.01, # 0.01 optuna: 0.005
        "lr_scheduler_type": 'cosine',
        "optim": 'adamw_torch', # adamw_torch, optuna: adamw_hf
        "gradient_accumulation_steps": 6, # 1,6
        "evaluation_strategy": 'epoch',
        "save_strategy": 'epoch',
        "save_total_limit": 5, # 6
        "fp16": True,
        "load_best_model_at_end": True,
        "seed": 42,
        "logging_dir": "./logs",
        "logging_strategy": "epoch",
        "predict_with_generate": True,
        "generation_max_length": 100, # 100
        "do_train": True,
        "do_eval": True,
        "early_stopping_patience": 5, # 3
        "early_stopping_threshold": 0, # 0.001,0
        "report_to": "wandb" # (선택) wandb를 사용할 때 설정합니다.
    },
    # (선택) wandb 홈페이지에 가입하여 얻은 정보를 기반으로 작성합니다.
    "wandb": {
        "entity": "upstage6_doc_classification",
        "project": "baseline_Pegasus",
        "name": "nlp"
    },
    "inference": {
        "ckt_path": "model ckt path", # 사전 학습이 진행된 모델의 checkpoint 경로를 설정합니다.
        "result_path": "./prediction/",
        "no_repeat_ngram_size": 2,
        "early_stopping": True,
        "generate_max_length": 200, # 200 생성 최대 길이
        "num_beams": 4,
        "batch_size" : 8, # 16,32
        # 정확한 모델 평가를 위해 제거할 불필요한 생성 토큰들을 정의합니다.
        "remove_tokens": ['<usr>', f"{tokenizer.bos_token}", f"{tokenizer.eos_token}", f"{tokenizer.pad_token}"]
    }
}

- 참고✅    
: wandb 라이브러리를 사용하기 위해선 entity, project, name를 지정해주어야 합니다. wandb 홈페이지에 가입한 후 얻은 정보를 입력하여 작동할 수 있습니다.

In [8]:
# 모델의 구성 정보를 YAML 파일로 저장합니다.
config_path = "./config.yaml"
with open(config_path, "w") as file:
    yaml.dump(config_data, file, allow_unicode=True)

### 3) Configuration 불러오기

In [9]:
# 저장된 config 파일을 불러옵니다.
config_path = "./config.yaml"

with open(config_path, "r") as file:
    loaded_config = yaml.safe_load(file)

# 불러온 config 파일의 전체 내용을 확인합니다.
pprint(loaded_config)

{'general': {'data_path': '../data_en/',
             'model_name': 'google/pegasus-xsum',
             'output_dir': './'},
 'inference': {'batch_size': 8,
               'ckt_path': 'model ckt path',
               'early_stopping': True,
               'generate_max_length': 200,
               'no_repeat_ngram_size': 2,
               'num_beams': 4,
               'remove_tokens': ['<usr>', 'None', '</s>', '<pad>'],
               'result_path': './prediction/'},
 'tokenizer': {'bos_token': 'None',
               'decoder_max_len': 100,
               'encoder_max_len': 512,
               'eos_token': '</s>',
               'special_tokens': ['#Person1#',
                                  '#Person2#',
                                  '#Person3#',
                                  '#PhoneNumber#',
                                  '#Address#',
                                  '#PassportNumber#',
                                  '#DateOfBirth#',
                                 

In [10]:
# 실험에 쓰일 데이터의 경로, 사용될 모델, 모델의 최종 출력 결과를 저장할 경로에 대해 확인합니다.
loaded_config['general']

{'data_path': '../data_en/',
 'model_name': 'google/pegasus-xsum',
 'output_dir': './'}

In [7]:
# 이곳에 사용자가 저장한 데이터 dir 설정하기
# loaded_config['general']['data_path'] = "data_path"

In [11]:
# 데이터 전처리를 하기 위해 tokenization 과정에서 필요한 정보들을 확인합니다.
loaded_config['tokenizer']

{'bos_token': 'None',
 'decoder_max_len': 100,
 'encoder_max_len': 512,
 'eos_token': '</s>',
 'special_tokens': ['#Person1#',
  '#Person2#',
  '#Person3#',
  '#PhoneNumber#',
  '#Address#',
  '#PassportNumber#',
  '#DateOfBirth#',
  '#SSN#',
  '#CardNumber#',
  '#CarNumber#',
  '#Email#',
  '#Person#',
  '#Person4#',
  '#Person5#',
  '#Person6#',
  '#Person7#']}

In [12]:
# 모델이 훈련 시 적용될 매개변수를 확인합니다.
loaded_config['training']

{'do_eval': True,
 'do_train': True,
 'early_stopping_patience': 5,
 'early_stopping_threshold': 0,
 'evaluation_strategy': 'epoch',
 'fp16': True,
 'generation_max_length': 100,
 'gradient_accumulation_steps': 6,
 'learning_rate': 1e-05,
 'load_best_model_at_end': True,
 'logging_dir': './logs',
 'logging_strategy': 'epoch',
 'lr_scheduler_type': 'cosine',
 'num_train_epochs': 20,
 'optim': 'adamw_torch',
 'overwrite_output_dir': True,
 'per_device_eval_batch_size': 32,
 'per_device_train_batch_size': 16,
 'predict_with_generate': True,
 'report_to': 'wandb',
 'save_strategy': 'epoch',
 'save_total_limit': 5,
 'seed': 42,
 'warmup_ratio': 0.1,
 'weight_decay': 0.01}

In [13]:
# 모델 학습 과정에 대한 정보를 제공해주는 wandb 설정 내용을 확인합니다.
loaded_config['wandb']

{'entity': 'upstage6_doc_classification',
 'name': 'nlp',
 'project': 'baseline_Pegasus'}

In [11]:
# (선택) 이곳에 사용자가 사용할 wandb config 설정
# loaded_config['wandb']['entity'] = "사용할 wandb repo name"
# loaded_config['wandb']['name'] = "사용할 wandb run의 name"
# loaded_config['wandb']['project'] = "사용할 wandb project name"

In [14]:
# 모델이 최종 결과를 출력하기 위한 매개변수 정보를 확인합니다.
loaded_config['inference']

{'batch_size': 8,
 'ckt_path': 'model ckt path',
 'early_stopping': True,
 'generate_max_length': 200,
 'no_repeat_ngram_size': 2,
 'num_beams': 4,
 'remove_tokens': ['<usr>', 'None', '</s>', '<pad>'],
 'result_path': './prediction/'}

### 4) 데이터 불러와서 확인해보기
- 실험에서 쓰일 데이터를 load하여 데이터의 구조와 내용을 살펴보겠습니다.
- Train, dev, test 순서대로 12457, 499, 250개 씩 데이터가 구성되어 있습니다.

In [15]:
# config에 저장된 데이터 경로를 통해 train과 validation data를 불러옵니다.
data_path = loaded_config['general']['data_path']

# train data의 구조와 내용을 확인합니다.
train_df = pd.read_csv(os.path.join(data_path,'train_en.csv'))
train_df.tail()

,id,dialogue,summary,topic
12455,train_12455,#Person1#: Excuse me. You are Mr. Green from M...,Tan Ling picks Mr. Green up who is easily reco...,pick up someone
12456,train_12456,#Person1#: Mister Ewing said we should show up...,#Person1# and #Person2# plan to take the under...,conference center
12457,train_12457,#Person1#: How can I help you today?\n#Person2...,#Person2# rents a small car for 5 days with th...,rent a car
12458,train_12458,#Person1#: You look a bit unhappy today. What'...,#Person2#'s mom lost her job. #Person2# hopes ...,job losing
12459,train_12459,"#Person1#: Mom, I'm flying to visit uncle Lee'...",#Person1# asks for #Person2#'s idea of packing...,baggage pack


In [16]:
# validation data의 구조와 내용을 확인합니다.
val_df = pd.read_csv(os.path.join(data_path,'validation_en.csv'))
val_df.tail()

,id,dialogue,summary,topic
495,dev_495,"#Person1#: Now that it's the new year, I've de...",#Person1# decides to stop smoking and come out...,the new year
496,dev_496,"#Person1#: You married Joe, didn't you? \n#Per...",#Person1# thought #Person2# married Joe. #Pers...,fall in love
497,dev_497,#Person1#: How can I help you mam?\n#Person2#:...,#Person2#'s car makes noises. #Person1# thinks...,noises
498,dev_498,"#Person1#: Hello, Amazon's customer service. H...",#Person2# calls Amazon's customer service beca...,a missing page
499,dev_499,#Person1#: I can't believe it's almost summer....,#Person2# tells #Person1# #Person2# is going t...,summer vacation


In [17]:
# test data의 구조와 내용을 확인합니다.
test_df = pd.read_csv(os.path.join(data_path,'test_en.csv'))
test_df.tail()

,id,dialogue,summary,topic
1495,test_498_2,#Person1#: Matthew? Hi!\n#Person2#: Steve! Hav...,Matthew and Steve meet after a long time. Stev...,finding a house
1496,test_498_3,#Person1#: Matthew? Hi!\n#Person2#: Steve! Hav...,Steve has been looking for a place to live. Ma...,find a house
1497,test_499_1,"#Person1#: Hey, Betsy, did you hear the great ...",Frank invites Besty to the party to celebrate ...,party invitation
1498,test_499_2,"#Person1#: Hey, Betsy, did you hear the great ...",Frank invites Betsy to the big promotion party...,promotion party invitation
1499,test_499_3,"#Person1#: Hey, Betsy, did you hear the great ...",Frank invites Betsy to his party for his promo...,party invitation


## 1. 데이터 가공 및 데이터셋 클래스 구축
- csv file 을 불러와서 encoder 와 decoder의 입력형태로 가공해줍니다.
- 가공된 데이터를 torch dataset class 로 구축하여 모델에 입력가능한 형태로 만듭니다.

In [18]:
import pandas as pd

class Preprocess:
    def __init__(self,
            bos_token: str,
            eos_token: str,
        ) -> None:

        self.bos_token = bos_token
        self.eos_token = eos_token

    @staticmethod
    # 실험에 필요한 컬럼을 가져옵니다.
    def make_set_as_df(file_path, is_train=True):
        df = pd.read_csv(file_path)
        if is_train:
            return df[['id', 'dialogue', 'summary']]
        else:
            return df[['id', 'dialogue']]

    # Pegasus 모델의 입력, 출력 형태를 맞추기 위해 전처리를 진행합니다.
    def make_input(self, dataset, is_train=True):
        prefix = "summarize: "
        
        # 인코더 입력을 처리합니다.
        encoder_input = dataset['dialogue'].apply(lambda x: prefix + str(x))

        if is_train:
            # 디코더 입력: BOS + summary
            decoder_input = dataset['summary'].apply(lambda x: self.bos_token + " " + str(x))
            
            # 디코더 출력: summary + EOS
            decoder_output = dataset['summary'].apply(lambda x: str(x) + self.eos_token)
            
            return encoder_input.tolist(), decoder_input.tolist(), decoder_output.tolist()
        else:
            # 테스트 데이터인 경우 디코더 입력에 BOS만 넣어줍니다.
            decoder_input = [self.bos_token] * len(dataset['dialogue'])
            return encoder_input.tolist(), list(decoder_input)


In [19]:
# Train에 사용되는 Dataset 클래스를 정의합니다.
class DatasetForTrain(Dataset):
    def __init__(self, encoder_input, decoder_input, labels): # len 삭제 후 
        self.encoder_input = encoder_input
        self.decoder_input = decoder_input
        self.labels = labels
        self.len = len(encoder_input['input_ids']) # 데이터 길이 자동 계산

    def __getitem__(self, idx):
        # 인코더 인풋 처리 
        item = {key: val[idx].clone().detach() for key, val in self.encoder_input.items()} 
        # 디코더 인풋 처리 
        item['decoder_input_ids'] = item['input_ids']
        # labels 처리 (보통 인코더 인풋과 같거나 일부 padding 값을 -100으로 설정)
        labels=self.labels['input_ids'][idx].clone().detach()
        labels[labels == 0] = -100 # 패딩 값 무시 
        item['labels'] = labels 
        return item

    def __len__(self):
        return self.len

# Validation에 사용되는 Dataset 클래스를 정의합니다.
class DatasetForVal(Dataset):
    def __init__(self, encoder_input, labels):
        self.encoder_input = encoder_input
        # 학습처럼 decoder_input_ids는 필요 없음
        self.labels = labels
        self.len = len(encoder_input['input_ids'])

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encoder_input.items()} 
        labels=self.labels['input_ids'][idx].clone().detach()
        labels[labels == 0] = -100 # 패딩 값 무시 
        item['labels'] = labels
        
        return item

    def __len__(self):
        return self.len

# Test에 사용되는 Dataset 클래스를 정의합니다.
class DatasetForInference(Dataset):
    def __init__(self, encoder_input):
        self.encoder_input = encoder_input
        self.len = len(encoder_input['input_ids'])

    def __getitem__(self, idx):
        # Encoder Input 처리
        item = {key: val[idx].clone().detach() for key, val in self.encoder_input.items()}
        
        return item

    def __len__(self):
        return self.len



In [20]:
# tokenization 과정까지 진행된 최종적으로 모델에 입력될 데이터를 출력합니다.
def prepare_train_dataset(config, preprocessor, data_path, tokenizer):
    train_file_path = os.path.join(data_path, 'train_en.csv')
    val_file_path = os.path.join(data_path, 'validation_en.csv')

    # train, validation에 대해 각각 데이터프레임을 구축합니다.
    train_data = preprocessor.make_set_as_df(train_file_path)
    val_data = preprocessor.make_set_as_df(val_file_path)

    # 인코더와 디코더 입력 및 출력을 생성합니다.
    encoder_input_train, decoder_input_train, decoder_output_train = preprocessor.make_input(train_data)
    encoder_input_val, decoder_input_val, decoder_output_val = preprocessor.make_input(val_data)

    # Train 데이터셋의 Tokenization
    tokenized_encoder_inputs = tokenizer(encoder_input_train, return_tensors="pt", padding=True,
                                         truncation=True, max_length=config['tokenizer']['encoder_max_len'])
    tokenized_decoder_inputs = tokenizer(decoder_input_train, return_tensors="pt", padding=True,
                                         truncation=True, max_length=config['tokenizer']['decoder_max_len'])
    tokenized_decoder_outputs = tokenizer(text_target=decoder_output_train, return_tensors="pt", padding=True,
                                          truncation=True, max_length=config['tokenizer']['decoder_max_len'])

    train_inputs_dataset = DatasetForTrain(tokenized_encoder_inputs, tokenized_decoder_inputs, tokenized_decoder_outputs, len(encoder_input_train))

    # Validation 데이터셋의 Tokenization (decoder_input 없이 처리)
    val_tokenized_encoder_inputs = tokenizer(encoder_input_val, return_tensors="pt", padding=True,
                                             truncation=True, max_length=config['tokenizer']['encoder_max_len'])
    val_tokenized_decoder_outputs = tokenizer(text_target=decoder_output_val, return_tensors="pt", padding=True,
                                              truncation=True, max_length=config['tokenizer']['decoder_max_len'])

    val_inputs_dataset = DatasetForVal(val_tokenized_encoder_inputs, val_tokenized_decoder_outputs)

    return train_inputs_dataset, val_inputs_dataset


## 2. Trainer 및 Trainingargs 구축하기
- Huggingface 의 Trainer 와 Training arguments를 활용하여 모델 학습을 일괄적으로 처리해주는 클래스를 정의합니다.

In [21]:
# import re
# from rouge import Rouge

def compute_metrics(config, tokenizer, pred):
    rouge = Rouge()
    predictions = pred.predictions
    labels = pred.label_ids

    # 패딩 토큰으로 대체
    predictions[predictions == -100] = tokenizer.pad_token_id
    labels[labels == -100] = tokenizer.pad_token_id

    # 디코딩된 예측값과 레이블
    decoded_preds = tokenizer.batch_decode(predictions, clean_up_tokenization_spaces=True)
    decoded_labels = tokenizer.batch_decode(labels, clean_up_tokenization_spaces=True)

    # 불필요한 생성 토큰 제거
    remove_tokens = config['inference']['remove_tokens']
    replaced_predictions = decoded_preds.copy()
    replaced_labels = decoded_labels.copy()
    
    for token in remove_tokens:
        # 정규식을 사용하여 여러 경우의 불필요한 토큰을 처리
        replaced_predictions = [re.sub(rf"\b{token}\b", " ", sentence) for sentence in replaced_predictions]
        replaced_labels = [re.sub(rf"\b{token}\b", " ", sentence) for sentence in replaced_labels]

    # 중복된 공백 제거
    replaced_predictions = [" ".join(sentence.split()) for sentence in replaced_predictions]
    replaced_labels = [" ".join(sentence.split()) for sentence in replaced_labels]

    # 디버깅용 출력 (필요 시 로깅으로 교체 가능)
    print('-'*150)
    print(f"PRED: {replaced_predictions[0]}")
    print(f"GOLD: {replaced_labels[0]}")
    print('-'*150)
    print(f"PRED: {replaced_predictions[1]}")
    print(f"GOLD: {replaced_labels[1]}")
    print('-'*150)
    print(f"PRED: {replaced_predictions[2]}")
    print(f"GOLD: {replaced_labels[2]}")

    # ROUGE 점수 계산
    results = rouge.get_scores(replaced_predictions, replaced_labels, avg=True)

    # F1-score를 사용하여 평가
    result = {key: value["f"] for key, value in results.items()}
    return result


In [22]:
# 학습을 위한 trainer 클래스와 매개변수를 정의합니다.
def load_trainer_for_train(config, generate_model, tokenizer, train_inputs_dataset, val_inputs_dataset):
    print('-'*10, 'Make training arguments', '-'*10,)
    # set training args
    training_args = Seq2SeqTrainingArguments(
                output_dir=config['general']['output_dir'], # model output directory
                overwrite_output_dir=config['training']['overwrite_output_dir'],
                num_train_epochs=config['training']['num_train_epochs'],  # total number of training epochs
                learning_rate=config['training']['learning_rate'], # learning_rate
                per_device_train_batch_size=config['training']['per_device_train_batch_size'], # batch size per device during training
                per_device_eval_batch_size=config['training']['per_device_eval_batch_size'],# batch size for evaluation
                warmup_ratio=config['training']['warmup_ratio'],  # number of warmup steps for learning rate scheduler
                weight_decay=config['training']['weight_decay'],  # strength of weight decay
                lr_scheduler_type=config['training']['lr_scheduler_type'],
                optim =config['training']['optim'],
                gradient_accumulation_steps=config['training']['gradient_accumulation_steps'],
                evaluation_strategy=config['training']['evaluation_strategy'], # evaluation strategy to adopt during training
                save_strategy =config['training']['save_strategy'],
                save_total_limit=config['training']['save_total_limit'], # number of total save model.
                fp16=config['training']['fp16'],
                load_best_model_at_end=config['training']['load_best_model_at_end'], # 최종적으로 가장 높은 점수 저장
                seed=config['training']['seed'],
                logging_dir=config['training']['logging_dir'], # directory for storing logs
                logging_strategy=config['training']['logging_strategy'],
                predict_with_generate=config['training']['predict_with_generate'], #To use BLEU or ROUGE score
                generation_max_length=config['training']['generation_max_length'],
                do_train=config['training']['do_train'],
                do_eval=config['training']['do_eval'],
                report_to=config['training']['report_to'] # (선택) wandb를 사용할 때 설정합니다.
            )

    # (선택) 모델의 학습 과정을 추적하는 wandb를 사용하기 위해 초기화 해줍니다.
    wandb.init(
        entity=config['wandb']['entity'],
        project=config['wandb']['project'],
        name=config['wandb']['name'],
    )

    # (선택) 모델 checkpoint를 wandb에 저장하도록 환경 변수를 설정합니다.
    os.environ["WANDB_LOG_MODEL"]="end"
    os.environ["WANDB_WATCH"]="false"
    
    # Validation loss가 더 이상 개선되지 않을 때 학습을 중단시키는 EarlyStopping 기능을 사용합니다.
    MyCallback = EarlyStoppingCallback(
        early_stopping_patience=config['training']['early_stopping_patience'],
        early_stopping_threshold=config['training']['early_stopping_threshold']
    )
    print('-'*10, 'Make training arguments complete', '-'*10,)
    print('-'*10, 'Make trainer', '-'*10,)

    # Trainer 클래스를 정의합니다.
    trainer = Seq2SeqTrainer(
        model=generate_model, # 사용자가 사전 학습하기 위해 사용할 모델을 입력합니다.
        args=training_args,
        train_dataset=train_inputs_dataset,
        eval_dataset=val_inputs_dataset,
        compute_metrics = lambda pred: compute_metrics(config, tokenizer, pred),
        callbacks = [MyCallback]
    )
    print('-'*10, 'Make trainer complete', '-'*10,)

    return trainer

In [23]:
# 학습을 위한 tokenizer와 사전 학습된 모델을 불러옵니다.
def load_tokenizer_and_model_for_train(config, device):
    print('-'*10, 'Load tokenizer & model', '-'*10,)
    print('-'*10, f'Model Name : {config["general"]["model_name"]}', '-'*10,)

    # 모델 및 토크나이저 이름 
    model_name = config['general']['model_name']

    # 모델 설정 불러오기
    model_config = PegasusConfig.from_pretrained(model_name)

    # 토크나이저 불러오기
    tokenizer = PegasusTokenizer.from_pretrained(model_name)

    # Special tokens 추가 
    special_tokens_dict={'additional_special_tokens':config['tokenizer']['special_tokens']}
    tokenizer.add_special_tokens(special_tokens_dict)

    # 사전 학습된 모델 불러오기 및 설정 적용
    generate_model = PegasusForConditionalGeneration.from_pretrained(config['general']['model_name'], config=model_config)
    
    # 토크나이저에 새로운 토큰이 추가되었으므로 임베딩 크기 재조정 
    generate_model.resize_token_embeddings(len(tokenizer)) 

    # GPU or CPU
    generate_model.to(device)

    # 모델 구성 정보 출력
    print('-'*10,'Model Configuration','-'*10)
    print(generate_model.config)

    print('-'*10, 'Load tokenizer & model complete', '-'*10,)
    return generate_model , tokenizer

## 3. 모델 학습하기

- 앞에서 구축한 클래스 및 함수를 활용하여 학습 진행합니다.

In [24]:
def main(config):
    try:
        # 사용할 device를 정의합니다.
        device = torch.device('cuda:0' if torch.cuda.is_available()  else 'cpu')
        print('-'*10, f'device : {device}', '-'*10,)
        print(torch.__version__)

        # 사용할 모델과 tokenizer를 불러옵니다.
        generate_model, tokenizer = load_tokenizer_and_model_for_train(config, device)
        print('-'*10,"tokenizer special tokens : ", tokenizer.special_tokens_map,'-'*10)
        print('-'*10,"additional special tokens : ", tokenizer.additional_special_tokens, tokenizer.additional_special_tokens_ids)

        # 학습에 사용할 데이터셋을 불러옵니다.
        preprocessor = Preprocess(config['tokenizer']['bos_token'], config['tokenizer']['eos_token']) # decoder_start_token: str, eos_token: str
        data_path = config['general']['data_path']
        train_inputs_dataset, val_inputs_dataset = prepare_train_dataset(config, preprocessor, data_path, tokenizer)

        # # Clear the cache to free up GPU memory
        # torch.cuda.empty_cache()

        # Trainer 클래스를 불러옵니다.
        trainer = load_trainer_for_train(config, generate_model, tokenizer, train_inputs_dataset, val_inputs_dataset)
        trainer.train()   # 모델 학습을 시작합니다.
    except Exception as e:
        print(f"Error during training:{e}")
    finally: 
        print('-'*10,'Training finished.CLosing WandB session','-'*10)

    # (선택) 모델 학습이 완료된 후 wandb를 종료합니다.
    wandb.finish()

In [25]:
if __name__ == "__main__":
    main(loaded_config)

---------- device : cuda:0 ----------
2.4.1+cu121
---------- Load tokenizer & model ----------
---------- Model Name : google/pegasus-xsum ----------


/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/259 [00:00<?, ?B/s]

---------- Model Configuration ----------
PegasusConfig {
  "_name_or_path": "google/pegasus-xsum",
  "activation_dropout": 0.1,
  "activation_function": "relu",
  "add_bias_logits": false,
  "add_final_layer_norm": true,
  "architectures": [
    "PegasusForConditionalGeneration"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 16,
  "decoder_start_token_id": 0,
  "do_blenderbot_90_layernorm": false,
  "dropout": 0.1,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 16,
  "eos_token_id": 1,
  "extra_pos_embeddings": 0,
  "force_bos_token_to_be_generated": false,
  "forced_eos_token_id": 1,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_encoder_decoder"

## 4. 모델 추론하기

In [ ]:
# 추론에 사용할 ckt 경로 설정
loaded_config['inference']['ckt_path'] = "checkpoint-22837"

- test data를 사용하여 모델의 성능을 확인합니다.

In [ ]:
# tokenization 과정까지 진행된 최종적으로 모델에 입력될 데이터를 출력합니다.
def prepare_test_dataset(config, preprocessor, tokenizer):

    test_file_path = os.path.join(config['general']['data_path'],'test.csv')

    test_data = preprocessor.make_set_as_df(test_file_path, is_train=False)
    test_id = test_data['fname']

    print('-'*150)
    print(f'test_data:\n{test_data["dialogue"][0]}')
    print('-'*150)

    encoder_input_test , decoder_input_test = preprocessor.make_input(test_data, is_test=True)
    print('-'*10, 'Load data complete', '-'*10,)

    test_tokenized_encoder_inputs = tokenizer(encoder_input_test, return_tensors="pt", padding=True,
                    add_special_tokens=True, truncation=True, max_length=config['tokenizer']['encoder_max_len'], return_token_type_ids=False,)
    
    test_encoder_inputs_dataset = DatasetForInference(test_tokenized_encoder_inputs, test_id, len(encoder_input_test))
    print('-'*10, 'Make dataset complete', '-'*10,)

    return test_data, test_encoder_inputs_dataset

In [ ]:
# 추론을 위한 tokenizer와 학습시킨 모델을 불러옵니다.
def load_tokenizer_and_model_for_test(config, device):
    print('-'*10, 'Load tokenizer & model', '-'*10,)

    model_name = config['general']['model_name']
    ckt_path = config['inference']['ckt_path']
    print('-'*10, f'Model Name : {model_name}', '-'*10,)

    tokenizer = PegasusTokenizer.from_pretrained(model_name)
    special_tokens_dict = {'additional_special_tokens': config['tokenizer']['special_tokens']}
    tokenizer.add_special_tokens(special_tokens_dict)

    generate_model = PegasusForConditionalGeneration.from_pretrained(ckt_path)
    generate_model.resize_token_embeddings(len(tokenizer))
    generate_model.to(device)
    print('-'*10, 'Load tokenizer & model complete', '-'*10,)

    return generate_model , tokenizer

In [ ]:
# 학습된 모델이 생성한 요약문의 출력 결과를 보여줍니다.
def inference(config):
    device = torch.device('cuda:0' if torch.cuda.is_available()  else 'cpu')
    print('-'*10, f'device : {device}', '-'*10,)
    print(torch.__version__)

    generate_model , tokenizer = load_tokenizer_and_model_for_test(config,device)

    preprocessor = Preprocess(config['tokenizer']['bos_token'], config['tokenizer']['eos_token'])

    test_data, test_encoder_inputs_dataset = prepare_test_dataset(config, preprocessor, tokenizer)
   
    batch_size=config['inference']['batch_size']
    dataloader = DataLoader(test_encoder_inputs_dataset, batch_size=batch_size)

    summary = []
    text_ids = []
    with torch.no_grad():
        for item in tqdm(dataloader):
            text_ids.extend(item['ID'])
            generated_ids = generate_model.generate(
                input_ids=item['input_ids'].to('cuda:0'),
                no_repeat_ngram_size=config['inference']['no_repeat_ngram_size'],
                early_stopping=config['inference']['early_stopping'],
                max_length=config['inference']['generate_max_length'],
                num_beams=config['inference']['num_beams'],
            )
            for ids in generated_ids:
                result = tokenizer.decode(ids,skip_special_tokens=True)
                summary.append(result)

    # 정확한 평가를 위하여 노이즈에 해당되는 스페셜 토큰을 제거합니다.
    # remove_tokens = config['inference']['remove_tokens']
    # preprocessed_summary = summary.copy()
    # for token in remove_tokens:
    #     preprocessed_summary = [sentence.replace(token," ") for sentence in preprocessed_summary]

    output = pd.DataFrame(
        {
            "fname": test_data['fname'],
            "summary" : summary,
        }
    )
    result_path = config['inference']['result_path']
    if not os.path.exists(result_path):
        os.makedirs(result_path)
    output.to_csv(os.path.join(result_path, "output_pegasus_en.csv"), index=False)

    return output

In [ ]:
# 학습된 모델의 test를 진행합니다.
if __name__ == "__main__":
    output = inference(loaded_config)

---------- device : cuda:0 ----------
2.1.0
---------- Load tokenizer & model ----------
---------- Model Name : lcw99/t5-large-korean-text-summary ----------
---------- Load tokenizer & model complete ----------
------------------------------------------------------------------------------------------------------------------------------------------------------
test_data:
#Person1#: 더슨 씨, 받아쓰기 좀 해주세요. 
#Person2#: 네, 실장님...
#Person1#: 이것은 오늘 오후까지 모든 직원에게 내부 메모로 전달되어야 합니다. 준비되셨나요?
#Person2#: 네, 실장님. 시작하셔도 됩니다.
#Person1#: 모든 직원들에게 주의하라... 즉시 효력을 발휘하여, 모든 사무실 통신은 이메일 통신과 공식 메모로 제한됩니다. 근무 시간 동안 직원들이 즉시 메시지 프로그램을 사용하는 것은 엄격히 금지됩니다.
#Person2#: 실장님, 이것은 내부 통신에만 적용되는 건가요? 아니면 외부 통신에도 제한이 되는 건가요?
#Person1#: 이것은 모든 통신에 적용되어야 합니다, 이 사무실 내의 직원들 사이뿐만 아니라 외부 통신에도 마찬가지입니다.
#Person2#: 하지만 실장님, 많은 직원들이 고객과 소통하기 위해 즉시 메시지를 사용하고 있습니다.
#Person1#: 그들은 그들의 의사소통 방법을 바꾸어야만 합니다. 이 사무실에서 누구도 즉시 메시지를 사용하지 않기를 원합니다. 너무 많은 시간을 낭비하게 됩니다! 이제, 메모를 계속해주세요. 우리가 어디까지 했나요?
#Person2#: 이것은 내부와 외부 통신에 적용됩니다.
#Person1#: 그렇습니다

100%|██████████| 125/125 [04:40<00:00,  2.25s/it]


In [ ]:
output  # 각 대화문에 대한 요약문이 출력됨을 확인할 수 있습니다.

,fname,summary
0,test_0,#Person1#은 모든 직원들에게 즉시 메시지 프로그램을 사용하지 않도록 주의하...
1,test_1,#Person2#는 교통 체증에 걸렸다. #Person1#은 대중교통을 이용하는 ...
2,test_2,#Person1#은 #Person2#에게 마샤와 히어로가 이혼을 신청했다고 말한다...
3,test_3,#Person1#은 브라이언의 생일 파티에 참석하고 그와 춤을 추며 축하한다. ...
4,test_4,#Person2#는 #Person1#에게 올림픽 스타디움이 6월에 완공될 예정이며...
...,...,...
494,test_495,잭은 찰리에게 비디오 게임을 하자고 제안한다. 찰리는 그것이 흥미롭다고 생각하고 ...
495,test_496,#Person2#는 #Person1#에게 컨트리 음악에 관심을 가지게 된 계기와 ...
496,test_497,"앨리스는 #Person1#에게 세탁기와 건조기를 어떻게 사용하는지 알려주고, 기계..."
497,test_498,스티브가 매튜에게 최근에 살 곳을 찾고 있다고 말한다. 그들은 함께 다우 부인의 ...


In [ ]:
len(output['summary'][0])

106